### <span style="color:red">To Do</span>
* Generate Predictions from 2021 class for analysis

# Get Similarity Between Different Recievers
Biggest use case is comparing rookies to current players to see what they might become.

In [1]:
import math
import time

import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [7]:
# Input data, skip top two rows
data = pd.read_csv('cfb_data_1_14_22_WR_year1.csv', index_col=0)
# Remove UDFA data because we are focused on finding the best players
data = data[(data['DP'] != "UDFA")]

In [8]:
data.head()

,Name,cfb_id,pfr_id,School,Conference,DR,DP,NFL POS,Draft Year,Designation in last Year,...,# of top 12 finishes,# of top 24 finishes,Year 1 TGT%,Year 1 EP,Year 1 reFPOE,Year 1 PPG,AVG EP Season 1-3,AVG PPG (PPR) Season 1-3,Year 1 Yards,Year 1 Yards/G
0,Xavier Worthy,xavier-worthy-1,NaN,Texas,Big 12,UNK,UNK,WR,2024,UNK,...,-,-,-,-,-,-,-,-,NaN,NaN
1,Mario Williams,mario-williams-2,NaN,Oklahoma,Big 12,UNK,UNK,WR,2024,UNK,...,-,-,-,-,-,-,-,-,NaN,NaN
2,Beaux Collins,beaux-collins-1,NaN,Clemson,ACC (Atlantic),UNK,UNK,WR,2024,UNK,...,-,-,-,-,-,-,-,-,NaN,NaN
3,Emeka Egbuka,emeka-egbuka-1,NaN,Ohio State,Big Ten (East),UNK,UNK,WR,2024,UNK,...,-,-,-,-,-,-,-,-,NaN,NaN
4,Troy Franklin,troy-franklin-2,NaN,Oregon,Pac-12 (North),UNK,UNK,WR,2024,UNK,...,-,-,-,-,-,-,-,-,NaN,NaN


In [9]:
# Drop non-relevant columns, and mostly null columns using fillna and value_counts
drop_list = ["cfb_id", "pfr_id", "Draft Year", "School", "Conference", "NFL POS", "Landing Team", "DOB", "Pre-Draft", 
             "Post-Draft", "Cr.Year BO", ">30%", "Cr.Year BO.1", "bench", "3Cone", "shuttle", "vert", "broadInches", "21", 
             "22", "23"]

# Drop yearly stats
for i in range(1, 24):
  drop_list.append("18." + str(i))
  drop_list.append("19." + str(i))
  drop_list.append("20." + str(i))
  drop_list.append("21." + str(i))
  drop_list.append("22." + str(i))
  drop_list.append("23." + str(i))

data = data.drop(drop_list, axis=1)

In [10]:
# Fill nulls

data = data.fillna(0)
data = data.replace("-", 0)
data = data.replace("<2 Yrs of Data", 0)

In [11]:
# Fill with averages

unk_list = ["WaSS", "HaSS", "BMI", "Decimal BOA (30%)"]
for i in unk_list:
  avg = (((data[i])[(data[i] != "UNK")]).astype(float)).mean()
  data[i] = data[i].replace("UNK", avg)

zero_list = ["Burst Score", "40", "height", "weight"]
for i in zero_list:
  avg = (((data[i])[(data[i] != 0)]).astype(float)).mean()
  data[i] = data[i].replace(0, avg)

In [65]:
# Split data into college players and NFL players

nfl_data = data[(data['DP'] != "UNK")]
nfl_names = nfl_data["Name"]
nfl_data = nfl_data.drop("Name", axis=1)

college_data = data[(data['DP'] == "UNK")]
college_names = list(college_data["Name"])
college_data = college_data.drop("Name", axis=1);

In [28]:
# Convert designation into int

nfl_data = nfl_data.replace("SR", 0)
nfl_data = nfl_data.replace("JR", 1)
nfl_data = nfl_data.replace("SO", 2)
nfl_data = nfl_data.replace("FR", 3)

college_data = college_data.replace("SR" , 0)
college_data = college_data.replace("UNK" , 0)
college_data = college_data.replace("JR" , 1)
college_data = college_data.replace("SO" , 2)
college_data = college_data.replace("FR" , 3)

In [29]:
# Convert all data to float and remove percentages

nfl_cols = nfl_data.columns
college_cols = college_data.columns

for col in nfl_cols:
  nfl_data[col] = nfl_data[col].astype(str).str.rstrip('%').astype(float) / 100.0
  nfl_data[col] = nfl_data[col].astype(float)

for col in college_cols:
    college_data[col] = college_data[col].astype(str).str.rstrip('%').astype(float) / 100.0

In [30]:
# Normalize data

nfl_data = (nfl_data-nfl_data.min())/(nfl_data.max()-nfl_data.min())
college_data = (college_data-college_data.min())/(college_data.max()-college_data.min())

In [31]:
nfl_data.head()

,DR,DP,Designation in last Year,Age IN DRAFT YEAR (9/1/dy),Conference Drafted Rate,Conference Hit Rate,Age Adjusted Hit Rate AVG,Pre-Draft Percentile,Post-Draft Percentile,>20%,...,# of top 12 finishes,# of top 24 finishes,Year 1 TGT%,Year 1 EP,Year 1 reFPOE,Year 1 PPG,AVG EP Season 1-3,AVG PPG (PPR) Season 1-3,Year 1 Yards,Year 1 Yards/G
54,0.333333,0.347656,0.5,0.142857,1.000000,0.435435,0.362398,0.808081,0.85,0.826087,...,0.0,0.0,0.158576,0.138918,0.368176,0.111022,0.145880,0.119147,0.093407,0.089073
59,0.166667,0.222656,0.5,0.142857,1.000000,0.435435,0.514986,0.707071,0.88,0.869565,...,0.0,0.0,0.213592,0.212089,0.232357,0.109018,0.222717,0.116829,0.095467,0.098255
60,0.166667,0.183594,1.0,0.142857,0.078652,0.000000,0.700272,0.919192,0.92,0.782609,...,0.0,0.0,0.449838,0.480912,0.374834,0.326253,0.505011,0.368104,0.299451,0.286501
66,0.833333,0.847656,0.5,0.142857,1.000000,0.435435,0.204360,0.818182,0.70,0.782609,...,0.0,0.0,0.100324,0.174973,0.380826,0.184369,0.183742,0.203987,0.024038,0.157025
67,0.166667,0.125000,0.5,0.142857,1.000000,0.435435,0.659401,0.979798,0.95,0.826087,...,0.0,0.0,0.621359,0.638388,0.373502,0.511423,0.670379,0.582290,0.370192,0.449954


In [32]:
# Drop NFL stats and Post-Draft stats
nfl_combine_cols = ["DR", "DP", "Post-Draft Percentile", "Calvin Johnson Index", "Combine Score", "BMI", "weight", "height", 
                    "40", "Burst Score", "HaSS", "WaSS" , nfl_data.columns[-10], nfl_data.columns[-2], nfl_data.columns[-3], 
                    nfl_data.columns[-4], nfl_data.columns[-5], nfl_data.columns[-6], nfl_data.columns[-7], nfl_data.columns[-8], 
                    nfl_data.columns[-9], nfl_data.columns[-11]]

# Drop NFL stats 
nfl_cols = [nfl_data.columns[-1], nfl_data.columns[-2], nfl_data.columns[-3], nfl_data.columns[-4], nfl_data.columns[-5], 
            nfl_data.columns[-6], nfl_data.columns[-7], nfl_data.columns[-8], nfl_data.columns[-9]]

nfl_data = nfl_data.drop(nfl_combine_cols, axis=1)
college_data = college_data.drop(nfl_combine_cols, axis=1)

In [33]:
nfl_data.head()

,Designation in last Year,Age IN DRAFT YEAR (9/1/dy),Conference Drafted Rate,Conference Hit Rate,Age Adjusted Hit Rate AVG,Pre-Draft Percentile,>20%,Decimal BOA (30%),Years Played,G,...,AVG MS REC Yds.1,PPG Above conference expectation (Best Year),S/EX (MS Yards Over Expectation using a 5 year conference AVG of top producer),TeamMate Score (TeamMate Over Expected),Production Score,Efficency Score (YTMpA Over conference AVG WR1s),18,19,20,Year 1 Yards/G
54,0.5,0.142857,1.000000,0.435435,0.362398,0.808081,0.826087,0.000000,0.50,0.396226,...,0.051682,0.330982,0.442771,0.433775,0.429319,0.240000,0.733333,0.666667,0.000000,0.089073
59,0.5,0.142857,1.000000,0.435435,0.514986,0.707071,0.869565,0.849643,0.75,0.528302,...,0.107934,0.678330,0.399096,0.294702,0.319372,0.306667,0.600000,0.800000,0.466667,0.098255
60,1.0,0.142857,0.078652,0.000000,0.700272,0.919192,0.782609,0.765645,0.25,0.377358,...,0.105051,0.722630,0.460843,0.182119,0.670157,0.306667,0.866667,0.266667,0.200000,0.286501
66,0.5,0.142857,1.000000,0.435435,0.204360,0.818182,0.782609,0.814364,0.75,0.622642,...,0.053545,0.395034,0.519578,0.433775,0.596859,0.280000,0.666667,0.800000,0.733333,0.157025
67,0.5,0.142857,1.000000,0.435435,0.659401,0.979798,0.826087,0.816044,0.75,0.584906,...,0.097335,0.864842,0.596386,0.301325,0.549738,0.573333,0.733333,0.800000,0.533333,0.449954


## First Year Yards Prediction

In [43]:
nfl_X = nfl_data.drop(["Year 1 Yards/G"], axis=1).to_numpy()
nfl_y = nfl_data["Year 1 Yards/G"].to_numpy()

In [46]:
college_X = college_data.drop(["Year 1 Yards/G"], axis=1).to_numpy()

In [56]:
features = list(college_data.columns)[:-1]

In [52]:
rf = RandomForestRegressor().fit(nfl_X, nfl_y)

In [58]:
# Feature importance for rookie year yards/G

feature_importance = rf.feature_importances_

feature_list = list()
for i in range(len(feature_importance)):
  feature_list.append((feature_importance[i], features[i]))

sorted_features = sorted(feature_list, reverse=True)
for i in sorted_features:
  print(i)

(0.3601305926486563, 'Age Adjusted Hit Rate AVG')
(0.037111864031810715, 'Conference Drafted Rate')
(0.03149003117151763, 'YPR')
(0.020033068332684187, 'Pre-Draft Percentile')
(0.01818217311958765, 'Conference Hit Rate')
(0.0179930446429056, 'PPG Above conference expectation (Best Year)')
(0.01760104487720145, 'First MS rec Yards.1')
(0.016620109038918, 'TeamMate Score (TeamMate Over Expected)')
(0.01644646744557101, 'AVG MS REC Yds.1')
(0.016381661836649266, 'last ms rec yards.1')
(0.015556635108268446, 'First Doa')
(0.014942756834693849, 'AVG PPG')
(0.013734833260633935, '19')
(0.01338639630507154, 'G')
(0.013165449349702756, 'Decimal BOA (30%)')
(0.012348984042774302, 'YARDS')
(0.012106296639485058, 'First Dominator.1')
(0.011984993865905878, 'Best MS rec Yards')
(0.01197826485979812, 'Best Dominator')
(0.01115107858929262, 'First RecYds/TmPatt Above AGE Avg')
(0.01114412845101171, 'Team')
(0.010850748010902543, 'First MS rec Yards')
(0.010151827238167366, 'rec')
(0.0098598496634060

In [53]:
rf.predict(college_X)

array([0.40337006, 0.1389348 , 0.24932048, 0.29044077, 0.14364555,
       0.15066116, 0.14173554, 0.21270891, 0.21626263, 0.2324977 ,
       0.16478421, 0.20802571, 0.2113315 , 0.32347107, 0.41651056,
       0.31924702, 0.28804408, 0.24741047, 0.2744169 , 0.31301194,
       0.36084481, 0.39741965, 0.33129477, 0.30701561, 0.10439853,
       0.16853994, 0.17380165, 0.23902663, 0.1596786 , 0.17209366,
       0.13889807, 0.11713499, 0.1894123 , 0.19730946, 0.19938476,
       0.19144169, 0.37125803, 0.15307622, 0.23408632, 0.20124885,
       0.15710744, 0.29808999, 0.29013774, 0.10923783, 0.34932048,
       0.18      , 0.14038567, 0.3789348 , 0.20963269, 0.24513315,
       0.3169146 , 0.13915519, 0.14309458, 0.28196511, 0.31690542,
       0.25264463, 0.37407713, 0.17104683, 0.29660239, 0.28501377,
       0.30472911, 0.2833517 , 0.31509642, 0.1637741 , 0.3017539 ,
       0.24556474, 0.30082645, 0.23722681])

In [64]:
college_names

0           Xavier Worthy
1          Mario Williams
2           Beaux Collins
3            Emeka Egbuka
4           Troy Franklin
              ...        
79       Tyrone Tracy Jr.
90         Jaivon Heiligh
105     Calvin Austin III
107         Jalen Tolbert
175    Demetris Robertson
Name: Name, Length: 68, dtype: object

In [68]:
# 2022 rookie yards/G preds

preds = rf.predict(college_X)

preds_list = list()
for i in range(len(preds)):
  preds_list.append((preds[i], college_names[i]))

sorted_preds = sorted(preds_list, reverse=True)
for i in sorted_preds:
  print(i)

(0.41651056014692367, 'Jordan Addison')
(0.40337006427915506, 'Xavier Worthy')
(0.39741965105601457, 'Dontay Demus')
(0.3789348025711661, 'David Bell')
(0.3740771349862259, 'Chris Olave')
(0.3712580348943985, 'Drake London')
(0.3608448117539026, 'Zay Flowers')
(0.3493204775022957, "Wan'Dale Robinson")
(0.3312947658402204, 'Marvin Mims')
(0.32347107438016537, 'Quentin Johnston')
(0.31924701561065194, 'Josh Downs')
(0.3169146005509643, 'Dante Wright')
(0.31690541781450887, 'Garrett Wilson')
(0.31509641873278255, 'Justyn Ross')
(0.3130119375573922, 'Kayshon Boutte')
(0.30701561065197436, 'Parker Washington')
(0.3047291092745638, 'Khalil Shakir')
(0.3017539026629937, 'Jaivon Heiligh')
(0.30082644628099187, 'Jalen Tolbert')
(0.2980899908172636, 'Jameson Williams')
(0.2966023875114784, 'Romeo Doubs')
(0.2904407713498622, 'Emeka Egbuka')
(0.290137741046832, 'George Pickens')
(0.2880440771349862, 'Jaxon Smith-Njigba')
(0.2850137741046833, 'Treylon Burks')
(0.2833516988062444, 'Erik Ezukanma')


## Getting Similarities

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
def similarity(rookies, data, in_nfl):
  row_labels = data.index.values
  data = data.to_numpy()

  rookie_labels = rookies.index.values
  rookies = rookies.to_numpy()

  output_dict = dict()

  for j in range(len(rookies)):
    rook = rookies[j]

    cos_similarity = list()
    for i in range(len(data)):
      cos_similarity.append((nfl_names.loc[row_labels[i]], int((1 - cosine(rook, data[i]))*100)))

    cos_similarity = sorted(cos_similarity, key = lambda x: x[1], reverse=True)

    if in_nfl:
      output_dict[nfl_names.loc[rookie_labels[j]]] = cos_similarity[1:11]
    else:
      output_dict[college_names.loc[rookie_labels[j]]] = cos_similarity[:10]
  return output_dict

In [ ]:
nfl_player_sim = similarity(nfl_data, nfl_data, True)


In [ ]:
nfl_player_sim['Davante Adams']

[('Allen Robinson', 98),
 ('Michael Crabtree', 98),
 ('Larry Fitzgerald', 98),
 ('Jabar Gaffney', 98),
 ('Courtland Sutton', 97),
 ('Jaelen Strong', 97),
 ('Marqise Lee', 97),
 ('Jeremy Maclin', 97),
 ('Sidney Rice', 97),
 ('James Hardy', 97)]

In [ ]:
#rookie_sim = similarity(college_data, nfl_data, False)
#rookie_sim_frame = pd.DataFrame(rookie_sim).transpose()

In [ ]:
rookie_sim_frame.to_csv('rookie_sim_2022.csv')
!cp rookie_sim_2022.csv "/content/gdrive/My Drive/Football_Projects/WR_Similarity/"